In [31]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate, Lambda
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras import backend as K
from keras.preprocessing.text import Tokenizer

In [52]:
df = pd.read_csv('haikus.csv', encoding='latin1')
df = df[df['lang'] == 'en'].copy()
df = df[~pd.isnull(df['0']) & ~pd.isnull(df['1']) & ~pd.isnull(df['2'])].copy()

max_line_len = max([max(df['%s' % i].str.len()) for i in range(3)])

for i in range(3):
    i = str(i)
    df['%s_in' % i] = df[i].str[0] + df[i].str.pad(max_line_len, 'right', '\n')
    df['%s_out' % i] = df[i].str.pad(max_line_len, 'right', '\n') + '\n'
    
max_line_len += 1

df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,haiku,lang,0_in,0_out,1_in,1_out,2_in,2_out
3,3,Memorial Day --,a shadow for each,white cross,NaN,NaN,NaN,NaN,NaN,NaN,Memorial Day --\r\na shadow for each\r\nwhite ...,en,MMemorial Day --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Memorial Day --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aa shadow for each\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a shadow for each\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,wwhite cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,white cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
4,4,spring rain -,as the doctor speaks,i think of lilacs,NaN,NaN,NaN,NaN,NaN,NaN,spring rain -\r\nas the doctor speaks\r\ni thi...,en,sspring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aas the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\...,as the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\n...,ii think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,i think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
5,5,spring moonset --,a rice ball for,breakfast,NaN,NaN,NaN,NaN,NaN,NaN,spring moonset --\r\na rice ball for\r\nbreakfast,en,sspring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aa rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,bbreakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,breakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
6,6,sunny afternoon,an old man lingers,near the mailbox,NaN,NaN,NaN,NaN,NaN,NaN,sunny afternoon\r\nan old man lingers\r\nnear ...,en,ssunny afternoon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,sunny afternoon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aan old man lingers\n\n\n\n\n\n\n\n\n\n\n\n\n\...,an old man lingers\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,nnear the mailbox\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,near the mailbox\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
7,7,cinco de mayo,horses roll,in the shallows,NaN,NaN,NaN,NaN,NaN,NaN,cinco de mayo\r\nhorses roll\r\nin the shallows,en,ccinco de mayo\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,cinco de mayo\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,hhorses roll\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,horses roll\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,iin the shallows\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,in the shallows\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
8,8,quitting time,the smell of rain,in the lobby,NaN,NaN,NaN,NaN,NaN,NaN,quitting time\r\nthe smell of rain\r\nin the l...,en,qquitting time\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,quitting time\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,tthe smell of rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,the smell of rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,iin the lobby\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,in the lobby\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
9,9,waves,slowly cresting towards shore,a faint moon,NaN,NaN,NaN,NaN,NaN,NaN,waves\r\nslowly cresting towards shore\r\na fa...,en,wwaves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,waves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,sslowly cresting towards shore\n\n\n\n\n\n\n\n...,slowly cresting towards shore\n\n\n\n\n\n\n\n\...,aa faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,a faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
10,10,overnight rain --,the scent of orange blossoms,in a desert town,NaN,NaN,NaN,NaN,NaN,NaN,overnight rain --\r\nthe scent of orange bloss...,en,oovernight rain --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,overnight rain --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,tthe scent of orange blossoms\n\n\n\n\n\n\n\n\...,the scent of orange blossoms\n\n\n\n\n\n\n\n\n...,iin a desert town\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,in a desert town\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
13,13,misty summer rain,calling pheasant,in Zen temple,NaN,NaN,NaN,NaN,NaN,NaN,misty summer rain\r\ncalling pheasant\r\nin Ze...,en,mmisty summer rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,misty summer rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,ccalling pheasant\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,calling pheasant\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,iin Zen temple\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,in Zen temple\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
14,14,day is done,poppies amidst,the dying grass,Na

(4914, 3)

In [72]:
inputs = df[['0_in', '1_in', '2_in']].values

t = Tokenizer(filters='', char_level=True)
t.fit_on_texts(inputs.flatten())
X = np_utils.to_categorical([
    t.texts_to_sequences(inputs[:,i]) for i in range(3)
])

outputs = df[['0_out', '1_out', '2_out']].values

#t = Tokenizer(filters='', char_level=True)
#t.fit_on_texts(outputs.flatten())
Y = np_utils.to_categorical([
    t.texts_to_sequences(outputs[:,i]) for i in range(3)
])

n_tokens = len(t.word_counts) + 1

In [73]:
X.shape

(3, 4914, 44, 63)

In [71]:
n_tokens

62

In [61]:
t.texts_to_sequences('\n')

[[1]]

In [62]:
t.texts_to_matrix('\n')

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [74]:
latent_dim = 128

# Keras Model

inputs = [Input(shape=(None, n_tokens)) for i in range(3)]
lstms = [LSTM(latent_dim, return_state=True, return_sequences=True) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[lstm_h[i-1], lstm_c[i-1]])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i])
        
denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

# Enforce number of chars per line
#counter = Lambda(lambda x: K.sum(K.cast(K.less(K.argmax(x, axis=2), 60), K.floatx()), axis=1, keepdims=True))
#counts = [counter(i) for i in outputs]

#model = Model(inputs, outputs + counts)

model = Model(inputs, outputs)

# Use categorical for the character outputs and MSE for the counts
#model.compile(optimizer='rmsprop', loss=['categorical_crossentropy']*3 + ['mean_squared_error']*3)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_4 (InputLayer)                                              (None, None, 63)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_4 (LSTM)                                                     [(None, None, 128), (None, 128), (None, 128 98304                   input_4[0][0]                                                 

In [76]:
model.fit([X[0], X[1], X[2]], [Y[0], Y[1], Y[2]], batch_size=128, epochs=10)

Epoch 1/10
4914/4914 [==============================] - 9s 2ms/step - loss: 5.2626 - dense_4_loss: 1.7071 - dense_5_loss: 1.9382 - dense_6_loss: 1.6173
Epoch 2/10
4914/4914 [==============================] - 7s 1ms/step - loss: 3.9518 - dense_4_loss: 1.2872 - dense_5_loss: 1.4838 - dense_6_loss: 1.1809
Epoch 3/10
4914/4914 [==============================] - 7s 1ms/step - loss: 3.7423 - dense_4_loss: 1.1970 - dense_5_loss: 1.4189 - dense_6_loss: 1.1264
Epoch 4/10
4914/4914 [==============================] - 7s 1ms/step - loss: 3.5749 - dense_4_loss: 1.1401 - dense_5_loss: 1.3575 - dense_6_loss: 1.0773
Epoch 5/10
4914/4914 [==============================] - 7s 1ms/step - loss: 3.4312 - dense_4_loss: 1.1028 - dense_5_loss: 1.2991 - dense_6_loss: 1.0293
Epoch 6/10
4914/4914 [==============================] - 7s 1ms/step - loss: 3.2847 - dense_4_loss: 1.0675 - dense_5_loss: 1.2385 - dense_6_loss: 0.9787
Epoch 7/10
4914/4914 [==============================] - 7s 1ms/step - loss: 3.1339 - den

In [122]:
model.fit([X[0], X[1], X[2]], [Y[0], Y[1], Y[2]], batch_size=128, epochs=20)

Epoch 1/20
4914/4914 [==============================] - 7s 1ms/step - loss: 2.7055 - dense_4_loss: 0.8918 - dense_5_loss: 1.0297 - dense_6_loss: 0.7840
Epoch 2/20
4914/4914 [==============================] - 7s 1ms/step - loss: 2.6376 - dense_4_loss: 0.8660 - dense_5_loss: 1.0054 - dense_6_loss: 0.7663
Epoch 3/20
4914/4914 [==============================] - 7s 2ms/step - loss: 2.5804 - dense_4_loss: 0.8433 - dense_5_loss: 0.9854 - dense_6_loss: 0.7518
Epoch 4/20
4914/4914 [==============================] - 7s 2ms/step - loss: 2.5309 - dense_4_loss: 0.8237 - dense_5_loss: 0.9682 - dense_6_loss: 0.7391
Epoch 5/20
4914/4914 [==============================] - 8s 2ms/step - loss: 2.4889 - dense_4_loss: 0.8057 - dense_5_loss: 0.9547 - dense_6_loss: 0.7286
Epoch 6/20
4914/4914 [==============================] - 8s 2ms/step - loss: 2.4508 - dense_4_loss: 0.7896 - dense_5_loss: 0.9417 - dense_6_loss: 0.7195
Epoch 7/20
4914/4914 [==============================] - 8s 2ms/step - loss: 2.4159 - den

In [129]:
model.fit([X[0], X[1], X[2]], [Y[0], Y[1], Y[2]], batch_size=128, epochs=50)

Epoch 1/50
4914/4914 [==============================] - 9s 2ms/step - loss: 2.0867 - dense_4_loss: 0.6288 - dense_5_loss: 0.8288 - dense_6_loss: 0.6291
Epoch 2/50
4914/4914 [==============================] - 8s 2ms/step - loss: 2.0704 - dense_4_loss: 0.6221 - dense_5_loss: 0.8237 - dense_6_loss: 0.6246
Epoch 3/50
4914/4914 [==============================] - 8s 2ms/step - loss: 2.0548 - dense_4_loss: 0.6155 - dense_5_loss: 0.8187 - dense_6_loss: 0.6207
Epoch 4/50
4914/4914 [==============================] - 9s 2ms/step - loss: 2.0394 - dense_4_loss: 0.6097 - dense_5_loss: 0.8136 - dense_6_loss: 0.6161
Epoch 5/50
4914/4914 [==============================] - 9s 2ms/step - loss: 2.0242 - dense_4_loss: 0.6037 - dense_5_loss: 0.8087 - dense_6_loss: 0.6117
Epoch 6/50
4914/4914 [==============================] - 9s 2ms/step - loss: 2.0099 - dense_4_loss: 0.5976 - dense_5_loss: 0.8042 - dense_6_loss: 0.6081
Epoch 7/50
4914/4914 [==============================] - 10s 2ms/step - loss: 1.9962 - de

In [178]:
model.fit([X[0], X[1], X[2]], [Y[0], Y[1], Y[2]], batch_size=128, epochs=50)

Epoch 1/50
4914/4914 [==============================] - 12s 2ms/step - loss: 1.5985 - dense_4_loss: 0.4552 - dense_5_loss: 0.6614 - dense_6_loss: 0.4819
Epoch 2/50
4914/4914 [==============================] - 12s 2ms/step - loss: 1.5923 - dense_4_loss: 0.4531 - dense_5_loss: 0.6593 - dense_6_loss: 0.4799
Epoch 3/50
4914/4914 [==============================] - 11s 2ms/step - loss: 1.5857 - dense_4_loss: 0.4513 - dense_5_loss: 0.6569 - dense_6_loss: 0.4774
Epoch 4/50
4914/4914 [==============================] - 11s 2ms/step - loss: 1.5792 - dense_4_loss: 0.4491 - dense_5_loss: 0.6551 - dense_6_loss: 0.4750
Epoch 5/50
4914/4914 [==============================] - 12s 3ms/step - loss: 1.5741 - dense_4_loss: 0.4475 - dense_5_loss: 0.6526 - dense_6_loss: 0.4740
Epoch 6/50
4914/4914 [==============================] - 13s 3ms/step - loss: 1.5666 - dense_4_loss: 0.4454 - dense_5_loss: 0.6501 - dense_6_loss: 0.4711
Epoch 7/50
4914/4914 [==============================] - 13s 3ms/step - loss: 1.561

In [130]:
generator_inputs = [Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)) for i in range(3)]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True) for i in range(3)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]

# Hook up line 0
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0]
)
generator_outputs = [generator_denses[0](generator_lstm_out[0])]
generator_models = [Model(generator_inputs[0], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])]

# Set weights for line 0
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())

generator_models[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (1, None, 63)             0         
_________________________________________________________________
lstm_16 (LSTM)               [(1, None, 128), (1, 128) 98304     
_________________________________________________________________
dense_16 (Dense)             (1, None, 63)             8127      
Total params: 106,431
Trainable params: 106,431
Non-trainable params: 0
_________________________________________________________________


In [131]:
generator_inputs_h = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
generator_inputs_c = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]

# Hook up lines 1 and 2
for i in [1, 2]:
    generator_lstm_out[i], generator_lstm_h[i], generator_lstm_c[i] = generator_lstms[i](
        generator_inputs[i], initial_state=[generator_inputs_h[i], generator_inputs_c[i]]
    )
    generator_outputs.append(generator_denses[i](generator_lstm_out[i]))

    generator_models.append(Model(
        [generator_inputs[i], generator_inputs_h[i], generator_inputs_c[i]],
        [generator_outputs[i], generator_lstm_h[i], generator_lstm_c[i]]
    ))

    generator_lstms[i].set_weights(lstms[i].get_weights())
    generator_denses[i].set_weights(denses[i].get_weights())

In [177]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

h,c = None, None
generator_models[0].reset_states()
for i in range(3):
    line = [random.randint(2,21)]
    
    def f():
        while True:
            if i > 0:
                yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
            else:
                yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
    while len(line) < max_line_len:
        character, h, c = generator_models[i].predict_generator(f(), steps=1)
        #line.append(np.argmax(character))
        line.append(sample(character[0][0], 2))
        print (np.argmax(character), line[-1])
        
    #print(line[1:line.index(1)])
    if 1 in line:
        print(t.sequences_to_texts([line[1:line.index(1)]]))
    else:
        print(t.sequences_to_texts([line[1:]]))

3 42
40 12
12 18
4 4
11 17
3 9
8 17
3 5
2 12
3 12
1 18
10 10
1 26
26 26
26 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
['< l u t c i c a l l u r . .']
5 8
2 14
6 6
4 8
16 3
2 27
7 4
2 4
3 23
2 2
12 15
5 6
10 31
2 2
4 9
8 4
2 7
1 2
9 16
10 18
10 20
1 26
2 41
1 3
1 19
1 3
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
["n m o n e ' t t y   f o x   i t s   g u p . ; e w e"]
18 18
8 8
13 12
3 9
8 7
4 40
3 6
8 13
1 9
8 8
16 13
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
['u n l i s > o d i n d']


In [164]:
sample(character[0][0], 1.2)

1

In [165]:
character[0][0]

array([  5.17583633e-15,   9.99999881e-01,   1.11329246e-09,
         1.21417951e-08,   3.06974758e-11,   1.06083448e-10,
         3.43690876e-09,   1.81737669e-09,   4.45773106e-12,
         2.90719093e-09,   6.16431850e-10,   5.13554824e-11,
         5.77356207e-09,   1.47960498e-11,   2.86417916e-12,
         3.89426745e-13,   9.03020605e-14,   1.01231081e-11,
         1.39668112e-11,   1.00062779e-11,   5.88330686e-12,
         4.31039301e-11,   4.40172121e-09,   2.72105338e-10,
         5.37271356e-12,   2.13865672e-14,   1.83529991e-08,
         2.45146348e-10,   6.31345020e-14,   5.43910186e-13,
         1.52938863e-14,   3.69598856e-13,   1.48079492e-13,
         2.59105604e-09,   1.64144645e-13,   1.28964236e-13,
         8.55886828e-09,   4.43305948e-11,   8.52745999e-11,
         5.13704492e-08,   1.44564834e-13,   4.76275022e-15,
         7.16808662e-15,   6.35367376e-15,   6.24999354e-15,
         1.06181009e-14,   9.92755987e-15,   3.52043264e-13,
         1.72797754e-14,

In [157]:
character.shape

(1, 1, 63)

In [116]:
max_line_len

44

In [94]:
sorted(t.word_counts.items(), key=lambda x: -x[1])

[('\n', 392828),
 (' ', 31564),
 ('e', 24009),
 ('t', 19010),
 ('a', 17182),
 ('o', 16279),
 ('s', 16265),
 ('n', 15663),
 ('i', 14855),
 ('r', 13849),
 ('h', 11706),
 ('l', 9861),
 ('d', 7668),
 ('m', 6745),
 ('f', 6219),
 ('g', 6040),
 ('c', 5901),
 ('u', 5561),
 ('w', 5183),
 ('p', 3894),
 ('b', 3871),
 ('-', 3663),
 ('y', 3545),
 ('k', 2240),
 ('v', 1448),
 ('.', 1205),
 ("'", 667),
 ('z', 322),
 ('j', 300),
 ('q', 181),
 ('x', 173),
 ('\x97', 131),
 (',', 127),
 ('~', 55),
 (':', 48),
 ('?', 39),
 ('\x92', 38),
 ('"', 38),
 ('!', 30),
 ('>', 30),
 (';', 28),
 ('<', 22),
 ('2', 21),
 ('1', 21),
 ('0', 18),
 ('\x96', 17),
 ('3', 11),
 ('6', 11),
 ('5', 10),
 ('7', 8),
 ('8', 6),
 ('&', 6),
 ('é', 6),
 ('4', 6),
 ('\x85', 6),
 ('*', 5),
 ('9', 4),
 ('à', 3),
 ('(', 2),
 ('\xa0', 2),
 ('=', 1),
 ('ä', 1)]

In [120]:
t.sequences_to_texts([[6, 6, 8, 2, 4, 11, 3, 2, 7, 5, 10, 9, 8]])

['o o n   t h e   s a r i n']

In [111]:
np_utils.to_categorical(random.randint(2,21), num_classes=n_tokens).shape

(63,)

In [137]:
df[df['0'].str.contains('late night')]

,Unnamed: 0,0,1,2,3,4,5,6,7,8,haiku,lang,0_in,0_out,1_in,1_out,2_in,2_out
78,78,late night at the office,an exit sign floats,next to the moon,NaN,NaN,NaN,NaN,NaN,NaN,late night at the office\r\nan exit sign float...,en,llate night at the office\n\n\n\n\n\n\n\n\n\n\...,late night at the office\n\n\n\n\n\n\n\n\n\n\n...,aan exit sign floats\n\n\n\n\n\n\n\n\n\n\n\n\n...,an exit sign floats\n\n\n\n\n\n\n\n\n\n\n\n\n\...,nnext to the moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,next to the moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2258,2258,late night traffic,the swirl of leaves after,the last bus,NaN,NaN,NaN,NaN,NaN,NaN,late night traffic\r\nthe swirl of leaves afte...,en,llate night traffic\n\n\n\n\n\n\n\n\n\n\n\n\n\...,late night traffic\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,tthe swirl of leaves after\n\n\n\n\n\n\n\n\n\n...,the swirl of leaves after\n\n\n\n\n\n\n\n\n\n\...,tthe last bus\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,the last bus\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2302,2302,late night---,fireflies rise up,in the cemetery,NaN,NaN,NaN,NaN,NaN,NaN,late night---\r\nfireflies rise up\r\nin the c...,en,llate night---\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,late night---\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,ffireflies rise up\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,fireflies rise up\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,iin the cemetery\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,in the cemetery\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2655,2655,late night rain,the crickets finally,silent,NaN,NaN,NaN,NaN,NaN,NaN,late night rain\r\nthe crickets finally\r\nsilent,en,llate night rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,late night rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,tthe crickets finally\n\n\n\n\n\n\n\n\n\n\n\n\...,the crickets finally\n\n\n\n\n\n\n\n\n\n\n\n\n...,ssilent\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,silent\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2873,2873,late night drizzle---,she carries her belongings,in a kids cart,NaN,NaN,NaN,NaN,NaN,NaN,late night drizzle---\r\nshe carries her belon...,en,llate night drizzle---\n\n\n\n\n\n\n\n\n\n\n\n...,late night drizzle---\n\n\n\n\n\n\n\n\n\n\n\n\...,sshe carries her belongings\n\n\n\n\n\n\n\n\n\...,she carries her belongings\n\n\n\n\n\n\n\n\n\n...,iin a kids cart\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,in a kids cart\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2948,2948,late night---,she shows me her garden,I taste her tomatoes,NaN,NaN,NaN,NaN,NaN,NaN,late night---\r\nshe shows me her garden\r\nI ...,en,llate night---\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,late night---\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,sshe shows me her garden\n\n\n\n\n\n\n\n\n\n\n...,she shows me her garden\n\n\n\n\n\n\n\n\n\n\n\...,II taste her tomatoes\n\n\n\n\n\n\n\n\n\n\n\n\...,I taste her tomatoes\n\n\n\n\n\n\n\n\n\n\n\n\n...
3019,3019,late night on the beach---,the only light on,is that of the lighthouse,NaN,NaN,NaN,NaN,NaN,NaN,late night on the beach---\r\nthe only light o...,en,llate night on the beach---\n\n\n\n\n\n\n\n\n\...,late night on the beach---\n\n\n\n\n\n\n\n\n\n...,tthe only light on\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,the only light on\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,iis that of the lighthouse\n\n\n\n\n\n\n\n\n\n...,is that of the lighthouse\n\n\n\n\n\n\n\n\n\n\...
4274,4274,late night---,as city lights dim out,August full moon,NaN,NaN,NaN,NaN,NaN,NaN,late night---\r\nas city lights dim out\r\nAug...,en,llate night---\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,late night---\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aas city lights dim out\n\n\n\n\n\n\n\n\n\n\n\...,as city lights dim out\n\n\n\n\n\n\n\n\n\n\n\n...,AAugust full moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,August full moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4681,4681,late night at the airport,the shoe shine man,polishes his own shoes,NaN,NaN,NaN,NaN,NaN,NaN,late night at the airport\r\nthe shoe shine ma...,en,llate night at the airport\n\n\n\n\n\n\n\n\n\n...,late night at the airport\n\n\n\n\n\n\n\n\n\n\...,tthe shoe shine man\n\n\n\n\n\n\n\n\n\n\n\n\n\...,the shoe shine man\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,ppolishes his own s